# 🤖 AI YouTube Shorts — Google Colab Quickstart

**Fully automated 11-step pipeline**: Topic → Story → SEO → 5 Scene Images → TTS → SadTalker Avatar → Subtitles → Slideshow → YouTube Upload

📦 **Repository**: [github.com/tamilarasu18/ai-yt-automation](https://github.com/tamilarasu18/ai-yt-automation)

---

## ☀️ Daily Routine (After One-Time Setup)

```
1. Open this Colab notebook         → 5 seconds (bookmark it!)
2. Click "Runtime → Run all"        → 1 click
3. Wait ~15 minutes                 → go drink coffee ☕
4. Telegram notification pops       → "Video uploaded! ✅"
5. Close Colab                      → done for the day
```

> **That's it. 1 click. 15 minutes. Video live on YouTube.**

---

## 📋 Before You Start (One-Time Setup)

1. **Runtime**: Go to **Runtime → Change runtime type → T4 GPU**
2. **Google Drive** must have this structure:

```
MyDrive/
└── ai-youtube-automation/
    ├── config.json              ← Your settings (API keys, sheet URL, etc.)
    ├── client_secret.json       ← YouTube OAuth client (from GCP)
    ├── service_account.json     ← Google Sheets auth (from GCP)
    └── videos/                  ← Output videos saved here
```

3. **Google Sheet** must be shared with your service account email

---
## 🔧 Step 1 — Mount Google Drive

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 1: Mount Google Drive to access credentials
# ═══════════════════════════════════════════════════════════════

from google.colab import drive
drive.mount('/content/drive')

import os

# Verify Drive credentials exist
DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'

files_check = {
    'config.json': f'{DRIVE_BASE}/config.json',
    'service_account.json': f'{DRIVE_BASE}/service_account.json',
    'client_secret.json': f'{DRIVE_BASE}/client_secret.json',
}

print('📁 Checking Google Drive files...\n')
all_ok = True
for name, path in files_check.items():
    exists = os.path.exists(path)
    print(f'  {"✅" if exists else "❌"} {name}: {path}')
    if not exists:
        all_ok = False

if all_ok:
    print('\n✅ All credentials found in Google Drive!')
else:
    print('\n⚠️  Missing files! Upload them to Drive/ai-youtube-automation/ first.')

---
## 📦 Step 2 — Clone Repo & Install Dependencies

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 2: Clone repo + Install everything
# ═══════════════════════════════════════════════════════════════

import os

# Clone the repository
if not os.path.exists('/content/ai-yt-automation'):
    !git clone https://github.com/tamilarasu18/ai-yt-automation.git /content/ai-yt-automation
else:
    !cd /content/ai-yt-automation && git pull

os.chdir('/content/ai-yt-automation')

# System dependencies
!apt-get update -qq && apt-get install -y -qq ffmpeg espeak-ng > /dev/null 2>&1

# Install project + all optional dependencies
!pip install -e ".[dev,api,kokoro]" -q

# GPU dependencies (Stable Diffusion, SadTalker)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q 2>/dev/null
!pip install diffusers transformers accelerate safetensors -q
!pip install moviepy==1.0.3 -q
!pip install gspread google-auth google-auth-oauthlib google-api-python-client -q
!pip install python-telegram-bot==20.7 -q
!pip install openai-whisper -q

print('\n✅ All Python dependencies installed!')

---
## 🤖 Step 3 — Install Ollama + SadTalker

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 3: Install Ollama (LLM) + SadTalker (Avatar)
# ═══════════════════════════════════════════════════════════════

import subprocess, time, urllib.request

# ── Ollama ──
!curl -fsSL https://ollama.com/install.sh | sh
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, start_new_session=True)
print('⏳ Waiting for Ollama server...')
time.sleep(5)

# Verify Ollama is running
for i in range(5):
    try:
        urllib.request.urlopen('http://localhost:11434/api/tags', timeout=3)
        print('✅ Ollama server running!')
        break
    except Exception:
        time.sleep(2)
        print(f'   Retrying... ({(i+1)*2}s)')

# Pull model (4b for Colab T4 GPU, 12b for A100)
!ollama pull gemma3:4b

# ── SadTalker ──
if not os.path.exists('/content/SadTalker'):
    !git clone -q https://github.com/OpenTalker/SadTalker.git /content/SadTalker

# Patch for numpy 2.0 compatibility
!sed -i 's/np.VisibleDeprecationWarning/DeprecationWarning/g' /content/SadTalker/src/face3d/util/preprocess.py 2>/dev/null; true

# Install SadTalker dependencies
!pip install -q face-alignment==1.3.5 kornia==0.6.12 yacs facexlib gfpgan basicsr dlib 2>/dev/null
!pip install -q -r /content/SadTalker/requirements.txt 2>/dev/null; true

# Download pretrained models
!cd /content/SadTalker && bash scripts/download_models.sh 2>/dev/null; true

os.chdir('/content/ai-yt-automation')
print('\n✅ Ollama + SadTalker ready!')

---
## ⚙️ Step 4 — Load Config from Drive & Set Environment

Loads `config.json`, `service_account.json`, and `client_secret.json` from your Google Drive.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 4: Load credentials from Google Drive + set env vars
# ═══════════════════════════════════════════════════════════════

import json, os, shutil

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
WORK_DIR = '/content/ai-yt-automation'

# ── Copy credential files from Drive ──
for fname in ['service_account.json', 'client_secret.json']:
    src = f'{DRIVE_BASE}/{fname}'
    dst = f'{WORK_DIR}/{fname}'
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f'  ✅ Copied {fname} from Drive')
    else:
        print(f'  ⚠️  {fname} not found in Drive')

# ── Load config.json ──
config_path = f'{DRIVE_BASE}/config.json'
config = {}
if os.path.exists(config_path):
    with open(config_path) as f:
        config = json.load(f)
    print(f'  ✅ Loaded config.json from Drive')
else:
    print(f'  ❌ config.json not found — using defaults')

# ── Set environment variables from config ──
env_mapping = {
    # Google Sheets
    'google_sheet_url': 'GOOGLE_SHEET_URL',
    'google_sheet_name': 'GOOGLE_SHEET_NAME',
    # YouTube
    'youtube_client_id': 'YOUTUBE_CLIENT_ID',
    'youtube_client_secret': 'YOUTUBE_CLIENT_SECRET',
    'youtube_refresh_token': 'YOUTUBE_REFRESH_TOKEN',
    # Telegram
    'telegram_bot_token': 'TELEGRAM_BOT_TOKEN',
    'telegram_chat_id': 'TELEGRAM_CHAT_ID',
    # Video
    'video_privacy': 'VIDEO_PRIVACY',
    'auto_upload_youtube': 'AUTO_UPLOAD_YOUTUBE',
}

for config_key, env_key in env_mapping.items():
    val = config.get(config_key, '')
    if val:
        os.environ[env_key] = str(val)

# ── Fixed paths (Colab-specific) ──
os.environ['GOOGLE_SERVICE_ACCOUNT_FILE'] = f'{WORK_DIR}/service_account.json'
os.environ['WORK_DIR'] = WORK_DIR
os.environ['OUTPUT_DIR'] = f'{WORK_DIR}/output'
os.environ['AVATAR_IMAGE_PATH'] = f'{WORK_DIR}/assets/images/avatar.png'
os.environ['SADTALKER_DIR'] = '/content/SadTalker'
os.environ['DRIVE_OUTPUT_FOLDER'] = config.get('drive_output_folder', f'{DRIVE_BASE}/videos')

# ── Engine selection ──
os.environ['OLLAMA_HOST'] = 'http://localhost:11434'
os.environ['OLLAMA_MODEL'] = 'gemma3:4b'
os.environ['TTS_ENGINE'] = 'edge'
os.environ['IMAGE_ENGINE'] = 'sd'
os.environ['SD_MODEL'] = 'stabilityai/stable-diffusion-2-1'
os.environ['ENABLE_FACE_ENHANCEMENT'] = 'true'
os.environ['WHISPER_MODEL_SIZE'] = 'medium'
os.environ['SDXL_INFERENCE_STEPS'] = '25'

# ── Create output directory ──
os.makedirs(os.environ['OUTPUT_DIR'], exist_ok=True)
os.makedirs(os.environ['DRIVE_OUTPUT_FOLDER'], exist_ok=True)

print(f'\n{"="*50}')
print('📋 Configuration Summary:')
print(f'  Sheet:     {os.environ.get("GOOGLE_SHEET_URL", "NOT SET")[:60]}...')
print(f'  Avatar:    {os.environ["AVATAR_IMAGE_PATH"]}')
print(f'  TTS:       {os.environ["TTS_ENGINE"]}')
print(f'  Images:    {os.environ["IMAGE_ENGINE"]} ({os.environ["SD_MODEL"]})')
print(f'  Ollama:    {os.environ["OLLAMA_MODEL"]}')
print(f'  Upload:    {os.environ.get("AUTO_UPLOAD_YOUTUBE", "false")}')
print(f'  Drive out: {os.environ["DRIVE_OUTPUT_FOLDER"]}')
print(f'{"="*50}')
print('\n✅ Environment configured!')

---
## ✅ Step 5 — Verify Everything

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 5: Verify all components are working
# ═══════════════════════════════════════════════════════════════

import os, subprocess
os.chdir('/content/ai-yt-automation')

print('🔍 System Check\n')

checks = []

# GPU
import torch
gpu = torch.cuda.is_available()
checks.append(('GPU', gpu, torch.cuda.get_device_name(0) if gpu else 'No GPU'))

# Ollama
try:
    r = subprocess.run(['ollama', 'list'], capture_output=True, text=True, timeout=5)
    checks.append(('Ollama', r.returncode == 0, 'Running'))
except Exception:
    checks.append(('Ollama', False, 'Not running'))

# FFmpeg
try:
    r = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True, timeout=5)
    checks.append(('FFmpeg', r.returncode == 0, 'Installed'))
except Exception:
    checks.append(('FFmpeg', False, 'Missing'))

# Files
for name, path in [
    ('Avatar', os.environ.get('AVATAR_IMAGE_PATH', '')),
    ('Service Account', os.environ.get('GOOGLE_SERVICE_ACCOUNT_FILE', '')),
    ('SadTalker', '/content/SadTalker/checkpoints'),
]:
    checks.append((name, os.path.exists(path), path.split('/')[-1]))

# Python packages
for pkg_name, import_name in [
    ('Edge TTS', 'edge_tts'),
    ('Diffusers', 'diffusers'),
    ('Whisper', 'whisper'),
    ('MoviePy', 'moviepy'),
]:
    try:
        __import__(import_name)
        checks.append((pkg_name, True, 'Installed'))
    except ImportError:
        checks.append((pkg_name, False, 'Missing'))

# Drive
drive_ok = os.path.exists('/content/drive/MyDrive')
checks.append(('Google Drive', drive_ok, 'Mounted' if drive_ok else 'Not mounted'))

# Results
for name, ok, detail in checks:
    icon = '✅' if ok else '❌'
    print(f'  {icon} {name:18s} {detail}')

failed = sum(1 for _, ok, _ in checks if not ok)
print(f'\n{"="*50}')
if failed == 0:
    print('🚀 All checks passed! Ready to run the pipeline.')
else:
    print(f'⚠️  {failed} check(s) failed. Fix issues above before running.')

---
## 🧪 Step 6 — Test Run (Quick Validation)

Runs a quick test: **Story → 5 Scene Images → Audio** (skips avatar, subtitles, upload)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 6: Test pipeline (story → images → audio only)
# ═══════════════════════════════════════════════════════════════

os.chdir('/content/ai-yt-automation')
!python -m ai_shorts run --mode test

---
## 🎬 Step 7 — Full Pipeline Run

Complete 11-step pipeline:
```
 1. Fetch topic        → Google Sheet
 2. Generate story     → Ollama (randomized styles)
 3. Generate SEO       → title, description, hashtags
 4. Generate 5 prompts → one image prompt per scene
 5. Generate 5 images  → Stable Diffusion (local GPU)
 6. Generate audio     → Edge TTS
 7. Animate avatar     → SadTalker (talking head)
 8. Generate subtitles → Whisper
 9. Compose slideshow  → MoviePy (5 images + avatar overlay)
10. Upload YouTube     → resumable + scheduler
11. Backup + Notify    → Google Drive + Telegram
```

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 7: Full pipeline 🚀
# ═══════════════════════════════════════════════════════════════

os.chdir('/content/ai-yt-automation')
!python -m ai_shorts run --mode full

---
## 📺 Step 8 — Preview & Download

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 8: Preview generated content
# ═══════════════════════════════════════════════════════════════

from IPython.display import Video, display, Image
from pathlib import Path
import glob

output_dir = Path('/content/ai-yt-automation/output')

# Show scene images
scene_images = sorted(glob.glob(str(output_dir / '*/scenes/scene_*.png')))
if scene_images:
    print(f'🖼️  {len(scene_images)} Scene Images Generated:\n')
    for img in scene_images:
        display(Image(filename=img, width=250))

# Show final video
videos = sorted(glob.glob(str(output_dir / '*/final_video.mp4')))
if videos:
    print(f'\n🎬 Final Video: {videos[-1]}')
    display(Video(videos[-1], width=360))
else:
    # Check for avatar video (test mode)
    avatars = sorted(glob.glob(str(output_dir / '*/avatar.mp4')))
    if avatars:
        print(f'\n🗣️ Avatar Video (test mode): {avatars[-1]}')
        display(Video(avatars[-1], width=360))
    else:
        print('⚠️  No video found. Check pipeline output for errors.')

In [ ]:
# ═══════════════════════════════════════════════════════════════
# Download video to your local machine
# ═══════════════════════════════════════════════════════════════

from google.colab import files
import glob

videos = sorted(glob.glob('/content/ai-yt-automation/output/*/final_video.mp4'))
if videos:
    # Also copy to Drive
    import shutil
    drive_out = os.environ.get('DRIVE_OUTPUT_FOLDER', '/content/drive/MyDrive/ai-youtube-automation/videos')
    os.makedirs(drive_out, exist_ok=True)
    shutil.copy(videos[-1], f'{drive_out}/latest_video.mp4')
    print(f'💾 Saved to Drive: {drive_out}/latest_video.mp4')
    files.download(videos[-1])
else:
    print('⚠️  No video to download')

---
## ⚡ Bonus — Scheduled Upload

In [ ]:
# ═══════════════════════════════════════════════════════════════
# BONUS: Run with scheduled YouTube upload
# ═══════════════════════════════════════════════════════════════

os.environ['AUTO_UPLOAD_YOUTUBE'] = 'true'

# Schedule for a specific date/time (ISO 8601)
!python -m ai_shorts run --mode full --schedule "2026-03-01T10:00:00+05:30"

---
## 🔑 One-Time: YouTube OAuth Token (run only once)

If you don't have a YouTube refresh token yet, run this cell **once** to generate it.
Then add `youtube_refresh_token` to your `config.json` in Drive.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ONE-TIME: Generate YouTube refresh token
# ═══════════════════════════════════════════════════════════════
# ⚠️  Only run this ONCE! Save the output to config.json.

!pip install google-auth-oauthlib google-api-python-client -q

from google_auth_oauthlib.flow import InstalledAppFlow
import json

client_secret_path = '/content/ai-yt-automation/client_secret.json'
if not os.path.exists(client_secret_path):
    client_secret_path = '/content/drive/MyDrive/ai-youtube-automation/client_secret.json'

flow = InstalledAppFlow.from_client_secrets_file(
    client_secret_path,
    ['https://www.googleapis.com/auth/youtube.upload']
)
creds = flow.run_local_server(port=0)

print('=' * 60)
print('🔑 SAVE THESE TO YOUR config.json in Google Drive:')
print('=' * 60)
print(f'"youtube_client_id": "{creds.client_id}",')
print(f'"youtube_client_secret": "{creds.client_secret}",')
print(f'"youtube_refresh_token": "{creds.refresh_token}"')
print('=' * 60)

---
## 📝 config.json Template

Create this file and upload to `Google Drive → ai-youtube-automation/config.json`:

```json
{
  "google_sheet_url": "https://docs.google.com/spreadsheets/d/YOUR_SHEET_ID/edit",
  "google_sheet_name": "Sheet1",
  "youtube_client_id": "YOUR_CLIENT_ID.apps.googleusercontent.com",
  "youtube_client_secret": "GOCSPX-your-secret",
  "youtube_refresh_token": "1//your-refresh-token",
  "telegram_bot_token": "1234567890:AAxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "telegram_chat_id": "123456789",
  "auto_upload_youtube": true,
  "video_privacy": "public",
  "drive_output_folder": "/content/drive/MyDrive/ai-youtube-automation/videos"
}
```

---

## 🛠️ Troubleshooting

| Issue | Fix |
|-------|-----|
| `CUDA out of memory` | Runtime → Restart runtime, then run all again |
| `Ollama connection refused` | Re-run Step 3 to restart Ollama server |
| `SadTalker model not found` | Re-run Step 3 to download models |
| `Edge TTS timeout` | Set `TTS_ENGINE=kokoro` in Step 4 |
| `Whisper OOM` | Set `WHISPER_MODEL_SIZE=base` in Step 4 |
| `No topics found` | Add rows with Status=`Pending` in Google Sheet |
| `YouTube token expired` | Re-run the OAuth cell above |
| `Drive not mounted` | Re-run Step 1 |

### GPU Memory Reset
```python
import torch; torch.cuda.empty_cache()
```

---

**Built with Clean Architecture ❤️ — Your AI Employee Working 24/7**